In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import date
import seaborn as sns
import random

import matplotlib.pyplot as plt

from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.mixture import GaussianMixture


from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.stats.diagnostic import acorr_ljungbox

import scipy.stats as stats
from scipy.stats import probplot, laplace, norm, t


import statsmodels.api as sm
from statsmodels.nonparametric.kde import KDEUnivariate
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima_process import ArmaProcess

import pymc as pm
import pytensor.tensor as pt
import arviz as az

import tensorflow as tf
from tensorflow import keras


#from tensorflow.keras.utils import plot_model


######################################
#from pmdarima import auto_arima
#from diptest import diptest

In [ ]:
class Stochastic_prices_bayesian:
    def __init__(self, df, nfuture, last_price ,npath, rfr, risk_neutral = True):
        self.nfuture = nfuture
        self.df = df
        self.rfr = rfr
        self.risk_neutral = risk_neutral
        self.npath = npath
        self.last_price = last_price
        self.simulated_prices = None

        self.forced_loop = 2

    #Semi-analytical : Merton jump diffusion
    def simulate_future_prices_Merton(self, mu_samples, sigma_samples, mu_j_samples, sigma_j_samples, lambda_j_samples):
        simulated_prices = np.zeros((self.forced_loop, self.npath, self.nfuture + 1))

        simulated_prices[: , :, 0] = self.last_price  # Set initial price for all paths

        dt = 1 / 252  # Daily time step

        # Simulate using each posterior sample
        for iii in range(self.forced_loop):
            mu_sample = mu_samples[iii]
            sigma_sample = sigma_samples[iii]
            mu_j_sample = mu_j_samples[iii]
            sigma_j_sample = sigma_j_samples[iii]
            lamda_j_sample = lambda_j_samples[iii]

            # will be removed after checking for the bayesian posterior for this model
            mu_sample = 0.05
            sigma_sample = 0.15
            mu_j_sample = -0.01
            sigma_j_sample = 0.1
            lamda_j_sample = 1

            for ii in range(self.npath):
                    for t in range(1, self.nfuture + 1):
                        z = np.random.normal(0, 1, 1)
                        NN = np.random.poisson(lam=int(lamda_j_sample * dt), size=1)[0]
                        logJ_NN=np.random.normal(mu_j_sample, sigma_j_sample, NN)

                        j_first_term = -lamda_j_sample * (np.exp(mu_j_sample + 0.5 * sigma_j_sample**2) -1)
                        j_second_term = np.sum(logJ_NN)

                        if self.risk_neutral:
                            simulated_prices[iii,ii, t] = simulated_prices[iii,ii, t - 1] * np.exp((self.rfr - j_first_term - 0.5 * sigma_sample**2) * dt + sigma_sample * np.sqrt(dt) * z + j_second_term)
                        else:
                            simulated_prices[iii,ii, t] = simulated_prices[iii,ii, t - 1] * np.exp((mu_sample - 0.5 * sigma_sample**2) * dt + sigma_sample * np.sqrt(dt) * z + j_second_term)

            self.simulated_prices = np.mean(simulated_prices, axis=0)
        return self.simulated_prices

    #Analytical : Geometric brownian motion
    def simulate_future_prices_GBM(self, mu_samples, sigma_samples):
        simulated_prices = np.zeros((self.forced_loop, self.npath, self.nfuture + 1))

        simulated_prices[: , :, 0] = self.last_price  # Set initial price for all paths

        dt = 1 / 252  # Daily time step

        # Simulate using each posterior sample
        for iii in range(self.forced_loop):
            mu_sample = mu_samples[iii]
            sigma_sample = sigma_samples[iii]

            for ii in range(self.npath):
                    for t in range(1, self.nfuture + 1):
                        z = np.random.normal(0, 1, 1)
                        if self.risk_neutral:
                            simulated_prices[iii,ii, t] = simulated_prices[iii,ii, t - 1] * np.exp((self.rfr - 0.5 * sigma_sample**2) * dt + sigma_sample * np.sqrt(dt) * z)
                        else:
                            simulated_prices[iii,ii, t] = simulated_prices[iii,ii, t - 1] * np.exp((mu_sample - 0.5 * sigma_sample**2) * dt + sigma_sample * np.sqrt(dt) * z)
            self.simulated_prices = np.mean(simulated_prices, axis=0)
        return self.simulated_prices

    #Btree : COX-Ross-Rubinstein
    def simulate_future_prices_btree(self, mu_samples, sigma_samples):
        simulated_prices = np.zeros((self.forced_loop, self.npath, self.nfuture + 1))

        simulated_prices[:,:, 0] = self.last_price  # Set initial price for all paths

        dt = 1 / 252  # Daily time step

        # Simulate using each posterior sample
        for iii in range(self.forced_loop):
            mu_sample = mu_samples[iii]
            sigma_sample = sigma_samples[iii]

            u = np.exp(sigma_sample * np.sqrt(dt))
            d = 1 / u
            for ii in range(self.npath):
                    for t in range(1, self.nfuture + 1):
                        ur = np.random.rand()

                        if self.risk_neutral:
                            p = (np.exp(self.rfr * dt) - d) / (u - d)
                            move = ur < p  # True = up, False = down
                            simulated_prices[iii, ii, t] = simulated_prices[iii, ii, t - 1] * (u if move else d)
                        else:
                            p = (np.exp(mu_sample * dt) - d) / (u - d)
                            move = ur < p  # True = up, False = down
                            simulated_prices[iii, ii, t] = simulated_prices[iii, ii, t - 1] * (u if move else d)

        self.simulated_prices = np.mean(simulated_prices, axis=0)
        return self.simulated_prices

    def plot_monte_carlo(self, simulated_prices, split_index):
        print ("Today's value = ", self.df.iloc[-1])

        plt.figure(figsize=(12, 6))
        plt.plot(self.df[:], color='black', alpha=1, label="Historical Price")

        # Plot the first 100 paths
        for i in range(min(100, simulated_prices.shape[0])):
            plt.plot(self.df.index[split_index-1:], simulated_prices[i], color='gray', alpha=0.1, linewidth=0.5)

        std_path = np.std(simulated_prices, axis=0)
        mean_path = np.mean(simulated_prices, axis=0)

        plt.plot(self.df.index[split_index-1:], mean_path, color='blue', alpha=1, label="Mean Forecast")
        plt.fill_between(self.df.index[split_index-1:], mean_path - std_path, mean_path + std_path,
                     color='orange', alpha=0.2, label="±1 Std Dev")
        plt.axvline(x=self.df.index[split_index], color='k', linestyle='--', label='Train/Test split')

        plt.title(f"Simulated Future Stock Prices (Bayesian GBM)")
        plt.xlabel('Date')
        plt.ylabel('Price')
        plt.grid(True)
        plt.tight_layout()
        plt.legend()
        plt.show()

        final_prices = simulated_prices[:, -1]
        mu_final = np.mean(final_prices)
        std_final = np.std(final_prices)
        ci_lower, ci_upper = np.percentile(final_prices, [2.5, 97.5])

        plt.figure(figsize=(10, 5))

        # Histogram
        sns.histplot(final_prices, bins=50, kde=False, stat="density", color="skyblue", edgecolor="gray", label="Simulated Final Prices")

        # Fit and plot Gaussian
        x_vals = np.linspace(final_prices.min(), final_prices.max(), 200)
        pdf_vals = stats.norm.pdf(x_vals, mu_final, std_final)
        plt.plot(x_vals, pdf_vals, color='red', lw=2, label="Fitted Normal")

        # Mean line
        plt.axvline(mu_final, color='black', linestyle='--', label=f"Mean = {mu_final:.2f}")

        # ±1 Std Dev
        plt.axvspan(mu_final - std_final, mu_final + std_final, color='orange', alpha=0.2, label=f"±1 Std Dev")

        # 95% CI
        plt.axvline(ci_lower, color='blue', linestyle='--', alpha=0.8, label=f"95% CI = [{ci_lower:.2f}, {ci_upper:.2f}]")
        plt.axvline(ci_upper, color='blue', linestyle='--', alpha=0.8)

        plt.title("Distribution of Simulated Final Stock Prices")
        plt.xlabel("Final Price")
        plt.ylabel("Density")
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()
